In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from math import pi
# qiskit
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)
# qurry
sys.path.insert(0, './')
from backend import backendWrapper
from mori import TagMap
print("Modules import completed...")
print("-"*30+"\n ### Init IBMQ and Set up provider")
IBMQ.load_account()
IBMQ.providers()
provider = IBMQ.get_provider(
    hub='ibm-q-hub-ntu', group='ntu-internal', project='default')
backend = backendWrapper(provider)
print("IBMQ loading completed...")

Modules import completed...
------------------------------
 ### Init IBMQ and Set up provider
IBMQ loading completed...


In [12]:
map_x = 30
map_y = 30

cityMapList = [[(10, 10), (10, 20), (22, 20)]]
penalityList = [100, 1000, 10000]

### Prpepred function

In [13]:
def plot_map(my_map, my_path=None):
    xs = [x[0] for x in my_map]
    ys = [x[1] for x in my_map]
    plt.scatter(xs, ys, s=100)
    plt.xlim(0, map_x)
    plt.ylim(0, map_y)
#     plt.figure(figsize=(5,5))
    for i in range(len(my_map)):
        plt.text(xs[i]+2, ys[i]-3, i, fontsize = 20)
    
    if my_path:
        for i in range(len(my_path)-1):
            plt.plot([my_map[my_path[i]][0], my_map[my_path[i+1]][0]], [my_map[my_path[i]][1], my_map[my_path[i+1]][1]], 'ro-')
    plt.show()
    
def dist(p, q):
    return math.sqrt((float(p[0]) - float(q[0])) ** 2 + (float(p[1]) - float(q[1])) ** 2)

def to_distance_matrix(my_map):
    ret = []
    for i in range(len(my_map)):
        this_line = []
        for j in range(len(my_map)):
            this_line.append(dist(my_map[i], my_map[j]))
        ret.append(this_line)
            
    return ret

def brute_tsp(graph, s=0):
    vertex = []
    V = len(graph)
    for i in range(V):
        if i != s:
            vertex.append(i)

    min_path = maxsize
    next_permutation=permutations(vertex)
    for i in next_permutation:
        current_pathweight = 0
        k = s
        for j in i:
            current_pathweight += graph[k][j]
            k = j
        current_pathweight += graph[k][s]
        min_path = min(min_path, current_pathweight)
        if min_path == current_pathweight:
            ret_path = list(i)
    ret_path = [0] + ret_path + [0]
    return ret_path, min_path

def tsp_matrix_to_QUBO_matrix(w, penality=1000):
    n = len(w)
    Q = np.zeros((n*n, n*n))
    c = np.ones(n*n)
    A = np.max(w) * penality # penality
    c = c * (-4*A)

    # k, l is the index of Q
    # i, j, p, p_ is the index of w
    for k in range(n*n):
        for l in range(n*n):
            i = k // n
            j = l // n
            p = k % n 
            p_ = l % n 
            if  (p+1)%n == p_ and i != j:
                Q[k, l] = w[i, j] 
            elif p == p_:
                Q[k, l] = Q[k, l] + A
            elif i == j:
                Q[k, l] = Q[k, l] + A

    return Q, c

def QUBO_to_Ising(Q, c):
    J = Q/4
    h = (c + np.sum(Q, axis=1))/2

    for i in range(len(h)):
        J[i][i] += h[i]

    return J

### processing

In [14]:
distanceMatrixList = [to_distance_matrix(m) for m in cityMapList]

min_PD = [brute_tsp(distance) for distance in distanceMatrixList]
for m in min_PD:
    print(m)
    
    
QandCSet = {
    (i, f'p={p}'): tsp_matrix_to_QUBO_matrix(np.array(distance), p) 
for i, distance in enumerate(distanceMatrixList)
for p in penalityList}

IsingSet = {k: QUBO_to_Ising(v[0], v[1]) for k, v in QandCSet.items()}
IsingSet = {k: v.tolist() for k, v in IsingSet.items()}

([0, 2, 1, 0], 37.62049935181331)


In [15]:
IsingSet

{(0,
  'p=100'): [[1184.347701061905,
   390.5124837953327,
   390.5124837953327,
   390.5124837953327,
   2.5,
   0.0,
   390.5124837953327,
   3.905124837953327,
   0.0], [390.5124837953327,
   1184.347701061905,
   390.5124837953327,
   0.0,
   390.5124837953327,
   2.5,
   0.0,
   390.5124837953327,
   3.905124837953327], [390.5124837953327,
   390.5124837953327,
   1184.347701061905,
   2.5,
   0.0,
   390.5124837953327,
   3.905124837953327,
   0.0,
   390.5124837953327], [390.5124837953327,
   2.5,
   0.0,
   1182.5374513859983,
   390.5124837953327,
   390.5124837953327,
   390.5124837953327,
   3.0,
   0.0], [0.0,
   390.5124837953327,
   2.5,
   390.5124837953327,
   1182.5374513859983,
   390.5124837953327,
   0.0,
   390.5124837953327,
   3.0], [2.5,
   0.0,
   390.5124837953327,
   390.5124837953327,
   390.5124837953327,
   1182.5374513859979,
   3.0,
   0.0,
   390.5124837953327], [390.5124837953327,
   3.905124837953327,
   0.0,
   390.5124837953327,
   3.0,
   0.0,
   